### Multi Agent Collaborator
---

In this notebook, we create a multi-agent collaborator for the `Home networking` and `Doorbell configuration` agents.

`Multi-agent Collaboration` is a Amazon Bedrock Agents native capability that enables a hierarchical collaboration between agents. You can now enable agent collaboration and associate secondary agents to a supervisor one. These secondary agents can be any existing agent within the same account, including agents that have collaboration themselves. This composable pattern allows you to build a chain of agents, as shown in the figure below.

![multi-agent-arch](multi-agent-diagram-1.png)

### Setup

Make sure that your boto3 version is the latest one.

In [ ]:
!pip freeze | grep boto3

### Creating Agent

On this section we're going to declare global variables that will be act as helpers during entire notebook and you will start to create your agent.

In [ ]:
import os
import sys
import uuid
import boto3

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)
from globals import *

energy_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{MULTI_AGENT_NAME}'

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

In [ ]:
# Create the multi agent
multi_agent = agents.create_agent(
    MULTI_AGENT_NAME,
    """
        You are a home networking and doorbell configuration API expert. You are able to respond to user queries and provide the information to their questions.
    """,
    """
        You are a home networking and doorbell configuration API expert. You are able to respond to user queries and provide the information to their questions.
        You are able to perform actions and route requests to collaborator home networking and doorbell configuration sub agents that are responsible for returning you with the information on
        respective API specs and based on the user question.
        Resist the temptation to ask the user for input. Only do so after you have exhausted available actions. 
        Never ask the user for information that you already can retrieve yourself through available actions. 
    """,
    f"us.{BEDROCK_MODEL_NOVA_LITE}",
    agent_collaboration='SUPERVISOR'
)
multi_agent

In [ ]:
multi_agent_id = multi_agent[0]
%store multi_agent_id

In [ ]:
multi_agent_id

### Associate Collaborators

On this section, we're going to recover alias_id from previous agents (sub-agents) to add all of them inside energy one (which is multi-agent collaborator).

In [ ]:
%store -r
doorbell_agent_alias_arn, home_network_agent_alias_arn

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': home_network_agent_alias_arn,
        'sub_agent_instruction': """Route any home networking specific user queries to this agent. This agent specializes in home networking related questions.""",
        'sub_agent_association_name': 'HomeNetworkingCollaborator',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': doorbell_agent_alias_arn,
        'sub_agent_instruction': """Route any doorbell configuration specific user queries to this agent. This agent specializes in doorbell configuration related questions.""",
        'sub_agent_association_name': 'DoorbellConfigurationCollaborator',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

In [ ]:
sub_agents_list

In [ ]:
# associate the fmc and meraki sub agents to the main one
multi_agent_alias_id, multi_agent_alias_arn = agents.associate_sub_agents(
    multi_agent_id, sub_agents_list
)

In [ ]:
multi_agent_names = {
    f"{doorbell_agent_id}/{doorbell_agent_alias_id}": DOORBELL_AGENT_NAME,
    f"{home_network_agent_id}/{home_network_agent_alias_id}": HOME_NETWORK_AGENT_NAME
}

multi_agent_names

### Route to the Home Networking sub agent
---

In this portion of the notebook, we will ask the supervisor a question that needs to be routed to the home networking sub agent. The supervisor agent will lay out its plan, route it to the home networking sub agent and the sub agent will proceed with completing its tasks from there onwards.

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    """Is my living room camera online? My devideId is madhur2039""", 
    multi_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

Since the API specs that we use as a part of this solution are synthetically generated, this output will return an error with the URL. Replace the API spec with your custom spec and re-run this solution to get accurate results.

```{'execution_result': {'stdout': "Other error occurred: HTTPSConnectionPool(host='api.smarthomesecurity.example.com', port=443): Max retries exceeded with url: /v1/devices/cameras/madhur2039/status (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fbb3a043620>: Failed to establish a new connection: [Errno -2] Name or service not known'))\n", 'stderr': '', 'return_code': 0, 'success': True}}```

As given in the output above, the agent tries to call the API by executing the generated code, but receives an error because the API is synthetically generated.

### Route to both agents in parallel
---

In this portion of the notebook, we will ask the supervisor agent two questions. The supervisor agent will break the task into sub tasks and identify a plan. As a part of this plan, the request will be routed to the respective collaborator agents that will complete their associated sub tasks. As a part of this execution, there will be collaboration back and forth with the supervisor agent and the supervisor agent will return a human readable, consolidated result at the end.

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    """Is my living room camera online? My devideId is madhur2039. Also, I want to get email notifications for deliveries but push notifications for when someone rings the doorbell.""", 
    multi_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)